In [1]:
import requests
import time
import os
import sys
import re
import io
import json
import csv
import openpyxl
import pandas as pd
import numpy as np
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Komoran, Okt
from nltk import Text
from pykospacing import spacing
import kss

In [2]:
browser = Chrome()

titles = []
links = []

date = input("시작날짜 입력(eg.20210920):")

#링크, 제목
for i in range(0,16):
    
    for j in range(1,3):
        browser.get("https://pann.nate.com/talk/ranking/d?stdt=" + str(date) + "&page=" + str(j))
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        first_list = soup.find('div', {'class': 'cntList'}).findAll('li')

    for li in first_list: 
        f_link = li.findAll('a')
        for a in f_link:
            real_link = 'https://pann.nate.com' + a.get('href') 
        links.append(real_link)
    
    for li in first_list:
        f_title = li.findAll('dl')
        for dl in f_title:
            t = dl.find('a')
            real_title = t.get('title')
            real_title = re.sub('[^ ㄱ-ㅣ가-힣]','',real_title)
#             real_title = spacing(real_title)
#             real_title = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', real_title)
#             real_title = re.sub('[\xa0|0xed]', '', real_title)
#             real_title = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]','',real_title)
        titles.append(real_title)
        
    date = int(date) + 1

#print({'제목': titles, 'url': links}, sep='\n')

시작날짜 입력(eg.20210920):20210920


In [3]:
#본문
txt = []
    
for i in links:
    try: 
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None            
        html2 = res.text
        
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "viewarea"})            
        parags = contentArea.findAll("div", {"id" : "contentArea"})

        content = ""

        for parag in parags:
            content += parag.text
#         content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
#         content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
#         content = re.sub('[\xa0]', '', content)
#         content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = re.sub('[^ ㄱ-ㅣ가-힣]','',content)
        #content = spacing(content)
        
        txt.append(content)
             
    except HTTPError as e:
        txt.append('')
    except URLError as e:
        txt.append('')
    except AttributeError as e:
        txt.append('')
        
#print(txt)

In [4]:
#형태소 분석
from konlpy.tag import Okt

okt = Okt() 

title_morphs = []
txt_morphs = []
    
for i in titles:
    title_morphs.append(okt.morphs(i))
    
for i in txt:
    txt_morphs.append(okt.morphs(i))
    

nate_dict = {
    '제목' : titles,
    '본문' : txt
}

df = pd.DataFrame(nate_dict) 
df.to_csv('natepann_test.csv', index=False, encoding="utf-8-sig")

  
morphs_dict = {
    '제목 형태소' : title_morphs,
    '본문 형태소' : txt_morphs

}


df2 = pd.DataFrame(morphs_dict)
df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")

browser.close()

In [5]:
import pandas as pd
import re

document = pd.read_csv("natepann_Morphs.csv")
document = document["제목 형태소"].apply(str)+" "+document["본문 형태소"].apply(str)

print(document)
for i in range(len(document)):
    document[i] = re.sub('[^ ㄱ-ㅣ가-힣]','',document[i])
    
print(document)
document.to_csv('natapann_kiwi.txt', sep = '\t', index = False)

#명사

0           ['히틀러', '가', '대중', '들', '을', '다루던', '방법'] []
1      ['니', '들', '이', '태용', '얼굴', '깐다고'] ['태', '용이',...
2      ['아이즈', '원', '멤버', '들', '근황'] ['권', '은비', '팔로'...
3      ['나', '모르는', '동네', '에', '버려짐', '도와줘', '어떡해'] [...
4      ['남편', '이', '저', '보고', '식탐', '있다는데', '제', '가',...
                             ...                        
545    ['정말', '결혼', '생활', '이', '행복한', '분', '있나요'] ['판...
546    ['도련님', '에게', '존댓말', '쓰고', '처남', '에겐', '반말', '...
547               ['일본', '에', '생긴', '한국', '형', '편의점'] []
548    ['로제', '팬', '들', '이', '자기', '노래', '불러주니까', '울었...
549    ['설현', '이', '마른거냐', 'ㅋ'] ['제목', '살짝', '어그', '로...
Length: 550, dtype: object
0                                   히틀러 가 대중 들 을 다루던 방법 
1      니 들 이 태용 얼굴 깐다고 태 용이 못 생겨 질거 같음 ㅋㅋㅋㅋ 니 들 이 평생 ...
2      아이즈 원 멤버 들 근황 권 은비 팔로 우미 최근 에 솔로 데뷔 앨범 마무리 사쿠라...
3      나 모르는 동네 에 버려짐 도와줘 어떡해 엄마 가 만원 찾아놓은 거 가방 에 넣어 ...
4      남편 이 저 보고 식탐 있다는데 제 가 식탐 인지 봐주세요 거두 절미 하고 제목 처...
                             ...                        
545 

In [1]:
############### 추가 데이터 추출

In [2]:
import datetime
import time
import fasttext

start_time = time.time()
model_fname = 'result.bin'

model = fasttext.load_model(model_fname)
print("time : ", time.time() - start_time)

time :  8.041725158691406


In [3]:
model.get_nearest_neighbors("엔시티")

[(0.9137370586395264, '엔시'),
 (0.9132599234580994, '쓰레기'),
 (0.9114325642585754, '부모'),
 (0.9109306335449219, '대해'),
 (0.9108603596687317, '아버지'),
 (0.9107683300971985, '새아버지'),
 (0.9105382561683655, '여자친구'),
 (0.9104597568511963, '할아버지'),
 (0.9101191163063049, '부모님'),
 (0.9100772738456726, '생각')]

In [4]:
test = []
test = model.get_nearest_neighbors("엔시티")

test

[(0.9137370586395264, '엔시'),
 (0.9132599234580994, '쓰레기'),
 (0.9114325642585754, '부모'),
 (0.9109306335449219, '대해'),
 (0.9108603596687317, '아버지'),
 (0.9107683300971985, '새아버지'),
 (0.9105382561683655, '여자친구'),
 (0.9104597568511963, '할아버지'),
 (0.9101191163063049, '부모님'),
 (0.9100772738456726, '생각')]

In [5]:
def check_string(string):
    with open('SentiWord_Dict.txt','r', encoding='utf-8') as temp_f:
        datafile = temp_f.readlines()
    for line in datafile:
        if string in line:
            return True
    return False

for i in range(10):
    
    k = test[i][1]
    if check_string(k):
        print('True')
    else:
        print('False')

False
True
False
False
False
False
False
False
False
True


In [6]:
import re

def check_string(string):
    with open('SentiWord_Dict.txt','r', encoding='utf-8') as temp_f:
        datafile = temp_f.readlines()
    for line in datafile:
        if string in line:
            print(re.sub(r'[^0-9-?\d+]', '', line))
            return True
    return False

for i in range(10):
    
    k = test[i][1]
    check_string(k)

-1
-1


In [7]:
model.get_nearest_neighbors("스트릿우먼파이터")

[(0.8864094018936157, '스트릿'),
 (0.885384202003479, '생각'),
 (0.8852952718734741, '상황'),
 (0.8851924538612366, '제'),
 (0.8849736452102661, '준비'),
 (0.8849034905433655, '소리'),
 (0.8846746683120728, '대요'),
 (0.8845055103302002, '어머니'),
 (0.8844786286354065, '말'),
 (0.8844766616821289, '남동생')]

In [8]:
test = []
test = model.get_nearest_neighbors("스트릿우먼파이터")

test

[(0.8864094018936157, '스트릿'),
 (0.885384202003479, '생각'),
 (0.8852952718734741, '상황'),
 (0.8851924538612366, '제'),
 (0.8849736452102661, '준비'),
 (0.8849034905433655, '소리'),
 (0.8846746683120728, '대요'),
 (0.8845055103302002, '어머니'),
 (0.8844786286354065, '말'),
 (0.8844766616821289, '남동생')]

In [9]:
def check_string(string):
    with open('SentiWord_Dict.txt','r', encoding='utf-8') as temp_f:
        datafile = temp_f.readlines()
    for line in datafile:
        if string in line:
            return True
    return False

for i in range(10):
    
    k = test[i][1]
    if check_string(k):
        print('True')
    else:
        print('False')

False
True
True
True
False
True
False
False
True
False


In [10]:
import re

def check_string(string):
    with open('SentiWord_Dict.txt','r', encoding='utf-8') as temp_f:
        datafile = temp_f.readlines()
    for line in datafile:
        if string in line:
            print(re.sub(r'[^0-9-?\d+]', '', line))
            return True
    return False

for i in range(10):
    
    k = test[i][1]
    check_string(k)

-1
-2
-1
-1
-1
